In [3]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import numpy as np


import warnings
warnings.simplefilter('ignore', FutureWarning)

In [4]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.shape

(1030, 9)

In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [8]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [16]:
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 29ms/step - loss: 1662.2467 - val_loss: 1157.0575
Epoch 2/100
23/23 - 0s - 3ms/step - loss: 1529.7203 - val_loss: 1035.4871
Epoch 3/100
23/23 - 0s - 3ms/step - loss: 1306.5874 - val_loss: 842.4841
Epoch 4/100
23/23 - 0s - 3ms/step - loss: 973.1500 - val_loss: 601.3436
Epoch 5/100
23/23 - 0s - 3ms/step - loss: 613.2264 - val_loss: 377.1543
Epoch 6/100
23/23 - 0s - 3ms/step - loss: 363.7926 - val_loss: 249.9049
Epoch 7/100
23/23 - 0s - 3ms/step - loss: 271.2224 - val_loss: 204.6172
Epoch 8/100
23/23 - 0s - 3ms/step - loss: 245.7009 - val_loss: 182.9673
Epoch 9/100
23/23 - 0s - 3ms/step - loss: 226.2414 - val_loss: 177.7750
Epoch 10/100
23/23 - 0s - 3ms/step - loss: 213.0491 - val_loss: 172.5269
Epoch 11/100
23/23 - 0s - 3ms/step - loss: 202.9022 - val_loss: 169.2718
Epoch 12/100
23/23 - 0s - 3ms/step - loss: 194.9570 - val_loss: 167.7266
Epoch 13/100
23/23 - 0s - 3ms/step - loss: 187.7742 - val_loss: 164.4460
Epoch 14/100
23/23 - 0s - 3ms/step - loss: 182.1845 - 

In [17]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 40ms/step - loss: 1615.7505 - val_loss: 1019.6592
Epoch 2/100
23/23 - 0s - 3ms/step - loss: 918.6885 - val_loss: 475.9883
Epoch 3/100
23/23 - 0s - 3ms/step - loss: 351.6643 - val_loss: 265.0351
Epoch 4/100
23/23 - 0s - 3ms/step - loss: 253.1523 - val_loss: 222.3764
Epoch 5/100
23/23 - 0s - 3ms/step - loss: 221.7375 - val_loss: 231.7027
Epoch 6/100
23/23 - 0s - 3ms/step - loss: 198.6993 - val_loss: 214.3017
Epoch 7/100
23/23 - 0s - 3ms/step - loss: 179.3261 - val_loss: 193.4990
Epoch 8/100
23/23 - 0s - 3ms/step - loss: 163.6324 - val_loss: 200.8730
Epoch 9/100
23/23 - 0s - 3ms/step - loss: 150.2424 - val_loss: 179.6917
Epoch 10/100
23/23 - 0s - 3ms/step - loss: 136.1278 - val_loss: 164.9528
Epoch 11/100
23/23 - 0s - 3ms/step - loss: 123.7589 - val_loss: 163.3392
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 109.6805 - val_loss: 161.1154
Epoch 13/100
23/23 - 0s - 3ms/step - loss: 99.2267 - val_loss: 150.4337
Epoch 14/100
23/23 - 0s - 3ms/step - loss: 88.5077 - val_l

In [18]:
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.1, epochs=100, verbose=2)

Epoch 1/100
29/29 - 1s - 32ms/step - loss: 1430.8816 - val_loss: 819.2277
Epoch 2/100
29/29 - 0s - 3ms/step - loss: 483.2123 - val_loss: 293.9225
Epoch 3/100
29/29 - 0s - 3ms/step - loss: 236.1174 - val_loss: 217.4731
Epoch 4/100
29/29 - 0s - 3ms/step - loss: 202.8142 - val_loss: 192.6210
Epoch 5/100
29/29 - 0s - 2ms/step - loss: 182.0260 - val_loss: 179.7730
Epoch 6/100
29/29 - 0s - 3ms/step - loss: 162.3874 - val_loss: 160.4162
Epoch 7/100
29/29 - 0s - 3ms/step - loss: 145.1170 - val_loss: 150.3338
Epoch 8/100
29/29 - 0s - 2ms/step - loss: 133.9035 - val_loss: 130.9300
Epoch 9/100
29/29 - 0s - 3ms/step - loss: 119.0440 - val_loss: 124.8582
Epoch 10/100
29/29 - 0s - 3ms/step - loss: 107.2466 - val_loss: 110.7375
Epoch 11/100
29/29 - 0s - 3ms/step - loss: 92.7962 - val_loss: 100.6710
Epoch 12/100
29/29 - 0s - 3ms/step - loss: 82.2109 - val_loss: 99.6255
Epoch 13/100
29/29 - 0s - 3ms/step - loss: 76.2956 - val_loss: 77.7486
Epoch 14/100
29/29 - 0s - 3ms/step - loss: 67.4776 - val_loss: 